In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [7]:
# Importing the data

df = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
df.columns = ['Sentence', 'Score']

,Sentence,Score
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


Create a list of words that are in the negative and positive comments.

In [3]:
negative_words = pd.DataFrame([])
positive_words = pd.DataFrame([])
lower_no_punc = pd.DataFrame([])

for index in df.index:
    #print(df.iloc[index]['Score'])
    if df.iloc[index]['Score'] == 0:
        s = df.iloc[index]['Sentence']
        out = re.sub(r'[^\w\d\s]+', '', s)
        negative_words = negative_words.append(out.lower().split())

    if df.iloc[index]['Score'] == 1:
        s = df.iloc[index]['Sentence']
        out = re.sub(r'[^\w\d\s]+', '', s)
        positive_words = positive_words.append(out.lower().split())
        #lower_no_punc = lower_no_punc.append([out.lower()])
    lower_no_punc = lower_no_punc.append([out.lower()])

lower_no_punc = lower_no_punc.reset_index(drop=True)
lower_no_punc.columns = ['Sentence']
negative_words = pd.DataFrame(negative_words[0].unique())
positive_words = pd.DataFrame(positive_words[0].unique())

Dropping out the words that are in both positive and negative lists.

In [54]:
# Compare word lists to each other and remove matching.
# print(negative_words, positive_words)

neg_match = negative_words[0].isin(positive_words[0])
pos_match = positive_words[0].isin(negative_words[0])

for index in neg_match.index:
    if neg_match.iloc[index] == True:
        negative_words.drop(index, inplace=True)

for index in pos_match.index:
    if pos_match.iloc[index] == True:
        positive_words.drop(index, inplace=True)
        
negative_words = negative_words.reset_index(drop=True)
positive_words = positive_words.reset_index(drop=True)



AttributeError: 'str' object has no attribute 'isin'

Outputting 

In [ ]:
for index in lower_no_punc.index:
    if any(neg in lower_no_punc.loc[index]['Sentence'] for neg in negative_words[0]):
        df.loc[index, 'Neg'] = True
    #elif all(neg in lower_no_punc.loc[index]['Sentence'] for neg in negative_words[0]) == False:     
    if any(pos in lower_no_punc.loc[index]['Sentence'] for pos in positive_words[0]):
        df.loc[index, 'Pos'] = True
    #elif all(pos in lower_no_punc.loc[index]['Sentence'] for pos in positive_words[0]) == False:
    else:
        df.loc[index, 'Neg'] = False
        df.loc[index, 'Pos'] = False
        
df